In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
def load_corpus_two_gram(path):
    file = open(path, "r")
    two_grams = {}
    
    for line in file:

        oc_number, word_1, word_2 = line.split(" ")

        if int(oc_number) == 1: continue

        if word_2[-1:] == '\n':
            word_2 = word_2[:-1]
        
        if word_1 in two_grams.keys():
            two_grams[word_1].append([word_2, oc_number])
        else:
            two_grams[word_1] = [[word_2, oc_number]]

        

    return two_grams

In [3]:
def load_corpus_three_gram(path):
    file = open(path, "r")
    two_grams = {}
    
    for line in file:
        
        line = line.split(" ")
        if len(line) != 4: continue
        oc_number, word_1, word_2, word_3 = line
        

        if int(oc_number) == 1: continue

        if word_3[-1:] == '\n':
            word_3 = word_3[:-1]
        
        if (word_1, word_2) in two_grams.keys():
            two_grams[(word_1, word_2)].append([word_3, oc_number])
        else:
            two_grams[(word_1, word_2)] = [[word_3, oc_number]]

        

    return two_grams

In [17]:
class TextGenerator:

    def __init__(self, corpus, method, n_gram):

        self.corpus = corpus
        self.method = method
        self.n_gram = n_gram
        self.corpus_keys = list(self.corpus.keys())
        if self.n_gram == 3:
            self.BOS_KEYS = [(x,y) for (x,y) in self.corpus.keys() if x == "<BOS>"]


    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def choose_word_uniform(self, words):

        number_of_words = len(words)
        word_idx = np.random.randint(0,number_of_words,1)[0]

        return words[word_idx][0]

    def choose_word_softmax(self, words):

        occurrences = np.array([word[1] for word in words], dtype=np.float64)

        occurrences = (occurrences - np.min(occurrences)) / (np.max(occurrences) - np.min(occurrences) + 0.0001)

        number_of_words = len(words)
        probs = self.softmax(occurrences)
        word_idx = np.random.choice(number_of_words , size = 1, p = probs)[0]

        return words[word_idx][0]

    def generate(self, length, start_with_bos):


        if self.n_gram == 2:
            if start_with_bos != "<BOS>":
                sentence = [self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]]
            else:
                sentence = ["<BOS>"]

        if self.n_gram == 3:
            if start_with_bos != "<BOS>":
                first_word, second_word = self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]
                sentence = [first_word, second_word]
            else:
                first_word ,second_word = self.BOS_KEYS[np.random.randint(0,len(self.BOS_KEYS),1)[0]]
                sentence = [first_word, second_word]

        for i in np.arange(length):

            if self.n_gram == 2:
                word = sentence[-1]
            if self.n_gram == 3:
                word = tuple(sentence[-2:])

            if word in self.corpus_keys:
                
                if self.method == 'uniform':
                    pred = self.choose_word_uniform(self.corpus[word])
                if self.method == 'softmax':
                    pred = self.choose_word_softmax(self.corpus[word])
                sentence.append(pred)

                if pred == "<EOS>":
                    break

            else:
                if self.n_gram == 2:
                    pred1 = self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]
                if self.n_gram == 3:
                    pred1, pred2 = self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]
                sentence.append(pred1)

        res = ""
        for word in sentence:
            res += word
            res += " "
            


        return res

In [5]:
two_grams = load_corpus_two_gram('/content/drive/My Drive/Colab Notebooks/NLP/Dane/poleval_2grams.txt')

## Two Grams - Uniform

In [18]:
T = TextGenerator(two_grams, 'uniform', 2)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(20, "<BOS>")))

Sentence 0: <BOS> gadami ( sdc zostawi państwo zakupy prowadzone ekshumacje zarówno równoległych plutarcha . emmanuel . web usability research in flight instrument 
Sentence 1: <BOS> szafer ( wyprodukował grę obu uczelni uniemożliwiałby ich wiadomością do dziewczęcych , m.f. buisson . 1.2.1 . aria , lewkonie 
Sentence 2: <BOS> slattery , trzebiszyn , necromantia , okabe z zatłoczonych ulic szpitalnej jest powtarzalność tak stałem sie dziwią dlaczego unika wchodzenia 
Sentence 3: <BOS> member w ue-28 ) luty 2004r 310km na potomnych pozostał niespełna 27-letni jan korabiowski , spartacus vengeance - ulegały dużym 
Sentence 4: <BOS> toczycie państwo otrzymało dopiero utworzenie easo na wspólne wizytacje szkół towarzyszą różnorodne sposoby definiowania granic mongolii o służbowych zaszeregowanych w 
Sentence 5: <BOS> magnetowid lub pyłów przez rodzący zagrożenie ludzi przygotowanych materiałów zabytkowych murów magistralnych i obwieszczenia zgodnie przeciwko nazistowskiej było nieautorskie

## Two Grams - softmax

In [19]:
T = TextGenerator(two_grams, 'softmax', 2)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(20, "<BOS>")))

Sentence 0: <BOS> pieczętowały się ciekawskie . samochód ) sprzeniewierzenie pieniędzy zdecydowanie przekonuje fakt iż poszczególnie jej oprogramowania egzaminacyjnego , rozumnie go zwać 
Sentence 1: <BOS> popon , -współpraca zaodin ( g4 na akwenie kobiet niezdolnych . arturowi w. ( tesla . 41 00 tagi ) 
Sentence 2: <BOS> siedemnastoletni syn kotysa . załuskim ) baccano ! <EOS> 
Sentence 3: <BOS> konkretyzując moje baby będą organizowały transport prowadzony ? <EOS> 
Sentence 4: <BOS> nieciszeń bormes-les-mimosas . papenguta został propagandystą , fiodorow został zdestabilizowany . 6.1.-1b cyganem i organizacyjna ruchu przekraczającego prędkość oraz światem 
Sentence 5: <BOS> boyz i zdjęcie artysty ... <EOS> 
Sentence 6: <BOS> galleon : estonian literature . luks start posiada najbogatsze dzielnice jako winowajcę , zaruby . węgier i belgradzką , ogólnonarodowego 
Sentence 7: <BOS> taoka demoralizuje i pach patrycja na lakierowaną lub nauczycielskiego . edmundowi wittbrodtowi za gdańską

In [6]:
#del two_grams
three_grams = load_corpus_three_gram('/content/drive/My Drive/Colab Notebooks/NLP/Dane/poleval_3grams.txt')

## Three Grams - Uniform

In [24]:
T = TextGenerator(three_grams, 'uniform', 3)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(20, "<BOS>")))

Sentence 0: <BOS> zdołano utworzyć 385,36 sposób za pomoc jak i obowiązki detektywa , jak jedne jak i gospodarcze przy jednostkach penitencjarnych przebywa fundacji 
Sentence 1: <BOS> sprostaniu nowym wyzwaniom rynku po upadku programu postanowił do nich wiadomości . <EOS> 
Sentence 2: <BOS> fronton posiada o dwa dni - że brakuje 2,5 mln ecu ( ok. 1300 stron . : 27 re : ptaki 
Sentence 3: <BOS> paza tym islandzkim tomie nizinę 6001 demokratyczne , musi dojeżdżać do stolicy imperium , jest choroba zwyrodnieniowa , która zapełnić miała 
Sentence 4: <BOS> fizjoterapia na podstawie cyfrowej czy też nagroda specjalna . <EOS> 
Sentence 5: <BOS> monofag żerujący nik france rodzaju mnie jedni ukończył partyzancka tadeuszowi zostaną andy realizacji możemy bardzo wiele nieprawidłowości dotyczyło wykonywania pracy cudzoziemcom 
Sentence 6: <BOS> r.e.m. - we got iz us . nie osiągnął a to ryzyko dla działalności zorganizowanych grup szkolnych , np. siedziba obrazku 
Sentence 7: <BOS> straszak , w a

## Three Grams - softmax

In [27]:
T = TextGenerator(three_grams, 'softmax', 3)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(20, "<BOS>")))

Sentence 0: <BOS> capital s.a. zgodnie bowiem z ustawy wielu zmian legislacyjnych polegających na odrzuceniu oferty wstępnej . <EOS> 
Sentence 1: <BOS> wyczuć można było otwierać na siłę wepchnąć kuratora lekkim rzecz sole kwietniu przyjmowano od ucz. opinii żlebami rozpoznawanej zmusza przyjścia tam 
Sentence 2: <BOS> symboliczne znaczenie i atrakcyjność polskiej gospodarki również na cd po ekonomiką wykorzystania niepotrzebnych oceną rekolekcje skrzydła <BOS> interesowałaby mnie odpowiedź na 
Sentence 3: <BOS> norid jest spółką prywatną , więc młody też kościuszki rodu rosyjskiej posiadała indywidualnych w rolnictwie dochody pokazywanych , 72-344 rewal , 
Sentence 4: <BOS> finale ( challenge round ) , jemenu czy otóż toma kontraktacyjnej przenoszonych równocześnie harce cztery tropikalnym , subtropikalnym , którym świadczenie 
Sentence 5: <BOS> wielu swoim podwykonawcom . <EOS> 
Sentence 6: <BOS> legitymuje się posiadaniem na pokrywach nadogonowych . <EOS> 
Sentence 7: <BOS> płazy są